In [6]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [9]:
with open('input//intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [10]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [11]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

2022-05-16 13:15:11.108476: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 0s 2ms/step - loss: 2.0803 - accuracy: 0.0000e+00
Epoch 2/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0788 - accuracy: 0.0909
Epoch 3/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0783 - accuracy: 0.0909
Epoch 4/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0773 - accuracy: 0.1212
Epoch 5/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0765 - accuracy: 0.2121
Epoch 6/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0760 - accuracy: 0.2121
Epoch 7/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0756 - accuracy: 0.2121
Epoch 8/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0753 - accuracy: 0.2121
Epoch 9/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0748 - accuracy: 0.2121
Epoch 10/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0743 - accuracy: 0.2121
Epoch 11/500
2/2 [===================

In [13]:
# to save the trained model
model.save("input//chat_model")

import pickle

# to save the fitted tokenizer
with open('input//tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('input//label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

2022-05-16 13:15:50.208041: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: input//chat_model/assets
